In [1]:
# Import libraries
import numpy as np
import os

# Imports for using and authenticating BigQuery
from google.colab import auth

In [2]:
auth.authenticate_user()

In [3]:
# Replace project_id with your Google Cloud Project ID. 
os.environ["GOOGLE_CLOUD_PROJECT"]='amplified-asset-330216'


In [4]:
import ipywidgets as widgets

print("Variables for Region (Type 1) Queries")

gnomad_version_widget_region = widgets.Dropdown(
    options=['v2_1_1_exomes', 'v2_1_1_genomes', 'v3_genomes'],
    value='v3_genomes',
    description='gnomAD version:',
    disabled=False,
    style={'description_width': 'initial'}
)

display(gnomad_version_widget_region)

chromosome_widget_region = widgets.Dropdown(
    options=['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8',
             'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15',
             'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22',
             'chrX', 'chrY'],
    value='chr17',
    description='Chromosome:',
    disabled=False,
    style={'description_width': 'initial'}
)

display(chromosome_widget_region)

gene_symbol_widget_region= widgets.Text(
    value='BRCA1',
    placeholder='gene_symbol',
    description='Gene Symbol:',
    disabled=False,
    style={'description_width': 'initial'}
)

display(gene_symbol_widget_region)

Variables for Region (Type 1) Queries


Dropdown(description='gnomAD version:', index=2, options=('v2_1_1_exomes', 'v2_1_1_genomes', 'v3_genomes'), st…

Dropdown(description='Chromosome:', index=16, options=('chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7',…

Text(value='BRCA1', description='Gene Symbol:', placeholder='gene_symbol', style=DescriptionStyle(description_…

In [5]:
# Set the variables for the rest of the Type 1 queries based on the values above.
gnomad_version_region=gnomad_version_widget_region.value
chromosome_region=chromosome_widget_region.value
gene_symbol_region=gene_symbol_widget_region.value


print('Running Region (Type 1) queries on gnomAD version: {}, chromosome: {}, gene symbol: {}'.format(
    gnomad_version_region,
    chromosome_region,
    gene_symbol_region
))

if gnomad_version_region.startswith('v3'):
  # Variant type (snv, indel, multi-snv, multi-indel, or mixed) is stored under difference columns in V2 and V3
  variant_type_col = 'variant_type'
  extra_columns = ''
else:
  variant_type_col = 'alternate_bases. allele_type'
  # These vep columns only exist in V2
  extra_columns = 'vep.STRAND AS STRAND, vep.Protein_position AS Protein_pos,'


Running Region (Type 1) queries on gnomAD version: v3_genomes, chromosome: chr17, gene symbol: BRCA1


In [6]:
from google.cloud import bigquery

client = bigquery.Client()

def run_query(query, verbose=True):
    query_job = client.query(query)
    result = None
    if verbose :
      result = query_job.to_dataframe(progress_bar_type='tqdm_notebook')
      gb_processed = (query_job.total_bytes_billed / 1024 ** 3)
      print('This query processed {} GB of data which is {}% of your 1 TB monthly free quota.'.format(gb_processed, round(gb_processed / 1024 * 100, 4)))
    else :
      result = query_job.to_dataframe()
    return result


In [10]:

query_template = """
SELECT DISTINCT 
       start_position,
       reference_bases,
       alternate_bases.alt,
       {VAR_TYPE_COL} AS variant_type,
FROM `bigquery-public-data.gnomAD.{GNOMAD_VER}__{CHROM}` AS main_table,
     main_table.alternate_bases AS alternate_bases
WHERE start_position >= {X} AND start_position <= {Y}
ORDER BY 1,2
"""
query = query_template.format(GNOMAD_VER=gnomad_version_region,
                              CHROM="chr19",
                              VAR_TYPE_COL="variant_type", X=58345197-150, Y=58345197+150)

pas_var_df = run_query(query, verbose=False)
pas_var_df.head()


,start_position,reference_bases,alt,variant_type
0,58345054,G,A,snv
1,58345055,C,T,snv
2,58345057,T,G,snv
3,58345059,A,C,snv
4,58345063,T,C,snv


In [7]:
import pandas as pd


In [8]:

polyadb_bed = pd.read_csv("../content/polyadb_coordinates_utr3_hg38.bed", sep='\t', header=None, names=['chrom', 'pas_pos', 'end', 'gene', 'gene_id', 'strand'])


In [9]:

polyadb_bed


,chrom,pas_pos,end,gene,gene_id,strand
0,chr19,58346883,58346884,A1BG,A1BG.1,-
1,chr19,58345849,58345850,A1BG,A1BG.2,-
2,chr19,58345799,58345800,A1BG,A1BG.3,-
3,chr19,58345197,58345198,A1BG,A1BG.4,-
4,chr19,58343947,58343948,A1BG,A1BG.5,-
...,...,...,...,...,...,...
100095,chr1,77562501,77562502,ZZZ3,ZZZ3.6,-
100096,chr1,77562439,77562440,ZZZ3,ZZZ3.7,-
100097,chr1,77562106,77562107,ZZZ3,ZZZ3.8,-
100098,chr1,77559986,77559987,ZZZ3,ZZZ3.9,-


In [11]:

#chrs = ["chr22"]
#chrs = ["chr1", "chr2", "chr3", "chr4", "chr5"]
#chrs = ["chr6", "chr7", "chr8", "chr9", "chr10"]
#chrs = ["chrX", "chrY"]
#chrs = ["chr16", "chr17", "chr18", "chr19", "chr20", "chr21"]
chrs = ["chr11", "chr12", "chr13", "chr14", "chr15"]

for chr in chrs :

  print("chromosome = " + str(chr))

  polyadb_bed_chr = polyadb_bed.query("chrom == '" + chr + "'").copy().reset_index(drop=True)
  polyadb_bed_chr_gene = polyadb_bed_chr.groupby(['gene']).agg({"pas_pos" : ["min", "max"]}).copy().reset_index()
  polyadb_bed_chr_gene.columns = ['_'.join(col).strip() for col in polyadb_bed_chr_gene.columns.values]

  polyadb_bed_chr_gene['pas_pos_min'] -= 150
  polyadb_bed_chr_gene['pas_pos_max'] += 150

  print("n genes = " + str(len(polyadb_bed_chr_gene)))

  dfs_chr = []

  i = 0
  for _, row in polyadb_bed_chr_gene.iterrows() :

    gene = row['gene_']
    start = row['pas_pos_min']
    end = row['pas_pos_max']

    if i % 100 == 0 :
      print("[" + str(i) + "] Running query for gene = " + str(gene))
      print("[" + str(i) + "]  - interval = " + str([start, end]))

    query_template = """
    SELECT reference_name AS CHROM, 
          start_position AS POS,
          reference_bases AS REF,
          alternate_bases.alt AS ALT,
          AN,
          alternate_bases.AC AS AC,
          alternate_bases.AF AS AF,
          {VAR_TYPE_COL} AS VTYPE,
    FROM `bigquery-public-data.gnomAD.{GNOMAD_VER}__{CHROM}` AS main_table,
        main_table.alternate_bases AS alternate_bases
    WHERE start_position >= {X} AND start_position <= {Y} AND {VAR_TYPE_COL} = 'snv'
    ORDER BY 1,2
    """
    query = query_template.format(GNOMAD_VER=gnomad_version_region,
                                  CHROM=chr,
                                  VAR_TYPE_COL="variant_type", X=start, Y=end)

    pas_var_df = None
    if i % 100 == 0 :
      pas_var_df = run_query(query, verbose=True)
    else :
      pas_var_df = run_query(query, verbose=False)

    if i % 100 == 0 :
      print("[" + str(i) + "]  - n vars = " + str(len(pas_var_df)))

    dfs_chr.append(pas_var_df)

    i += 1
  
  final_df_chr = pd.concat(dfs_chr).copy().reset_index(drop=True)

  #Save chromosome dataframe
  print("n vars (total for chr) = " + str(len(final_df_chr)))

  final_df_chr.to_csv("polyadb_gnomad_v3_" + str(chr) + ".csv", sep='\t', index=False)


chromosome = chr11
n genes = 979
[0] Running query for gene = AAMDC
[0]  - interval = [77905849, 77906149]


Downloading:   0%|          | 0/40 [00:00<?, ?rows/s]

This query processed 0.009765625 GB of data which is 0.001% of your 1 TB monthly free quota.
[0]  - n vars = 40
[100] Running query for gene = C11orf1
[100]  - interval = [111883897, 111886106]


Downloading:   0%|          | 0/515 [00:00<?, ?rows/s]

This query processed 0.009765625 GB of data which is 0.001% of your 1 TB monthly free quota.
[100]  - n vars = 515
[200] Running query for gene = CNTN5
[200]  - interval = [100358705, 100359005]


...

This query processed 0.009765625 GB of data which is 0.001% of your 1 TB monthly free quota.
[300]  - n vars = 188
[400] Running query for gene = SCAPER
[400]  - interval = [76347775, 76348465]


Downloading:   0%|          | 0/106 [00:00<?, ?rows/s]

This query processed 0.009765625 GB of data which is 0.001% of your 1 TB monthly free quota.
[400]  - n vars = 106
[500] Running query for gene = ULK3
[500]  - interval = [74835985, 74836430]


Downloading:   0%|          | 0/86 [00:00<?, ?rows/s]

This query processed 0.009765625 GB of data which is 0.001% of your 1 TB monthly free quota.
[500]  - n vars = 86
n vars (total for chr) = 429147


In [ ]:
%cd

/root


In [ ]:
!ls

polyadb_gnomad_v3_chr22.csv


In [ ]:
gnomad_version_region

'v3_genomes'